<a href="https://colab.research.google.com/github/fresher96/counting-eye-blinks-in-a-video-stream/blob/master/EORA_test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from os import environ;
videoFile = environ.get('file', 0);

In [58]:
drive = 'E:/CLCs/drive/colab/';
# videoFile = 0;
# videoFile = drive + 'videos/offline-demo.mp4';

# drive = 'drive/My Drive/colab/';
# videoFile = drive + 'videos/offline-demo.mp4';


dlib_predictor = drive + 'shape_predictor_68_face_landmarks.dat';
outputPath = drive + 'output.mp4'
defaultFPS = 10

In [63]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

In [6]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear

In [7]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(dlib_predictor)

[INFO] loading facial landmark predictor...


In [8]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [42]:
class videoDownloader():
    """
    This class takes care of saving the resulting output video stream \
    on the hard disk
    """

    def __init__(self, fps = defaultFPS, totalNumberOfFrames = 'inf', path = outputPath):
        self.outputVideo = None;
        self.path = path;
        self.fps = fps;
        self.currentFrame = 0;
        self.totalNumberOfFrames = totalNumberOfFrames;

    def stop(self):
        self.outputVideo.release();

    def putFrame(self, frame):
        self.currentFrame += 1;
        if(self.currentFrame%25 == 0):
            print(f'processing frame {self.currentFrame} out of {self.totalNumberOfFrames}');

        if(self.outputVideo is None):
            height = frame.shape[0];
            width  = frame.shape[1];
            fourcc = cv2.VideoWriter_fourcc(*'XVID');
            self.outputVideo = cv2.VideoWriter(self.path, fourcc, self.fps, (width, height), 1);

        self.outputVideo.write(frame);

In [59]:
# start the video stream thread
print("[INFO] starting video stream thread...")

if(videoFile != 0):
#     vs = FileVideoStream(videoFile).start()
    capture = cv2.VideoCapture(videoFile)
    fileStream = True
    fps = capture.get(cv2.CAP_PROP_FPS)
    print(fps)
    tot = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    downloader = videoDownloader(fps, tot)
else:
#     vs = VideoStream(src=0).start()
    capture = cv2.VideoCapture(0)
    fileStream = False
    fps = defaultFPS;
    capture.set(cv2.CAP_PROP_FPS, fps)
    print(capture.get(cv2.CAP_PROP_FPS))
    downloader = videoDownloader(fps)
    
time.sleep(1.0)


# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 2
EYE_AR_CONSEC_FRAMES_ALERT = fps * 2; # 2 seconds of frames
# initialize the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0

[INFO] starting video stream thread...
10.0


In [60]:
# loop over frames from the video stream
while True:
    # if this is a file video stream, then we need to check if
    # there any more frames left in the buffer to process
#     if fileStream and not vs.more():
#         break
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale
    # channels)
#     frame = vs.read()

    status, frame = capture.read();

    if(not status):
        break;

    frame = imutils.resize(frame, width=800)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale frame
    rects = detector(gray, 0)


    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0

        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull],  -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)


        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        # otherwise, the eye aspect ratio is not below the blink
        # threshold
        else:
            # if the eyes were closed for a sufficient number of
            # then increment the total number of blinks
            if EYE_AR_CONSEC_FRAMES <= COUNTER and COUNTER < EYE_AR_CONSEC_FRAMES_ALERT:
                TOTAL += 1
            # reset the eye frame counter
            COUNTER = 0
            
        alert = COUNTER >= EYE_AR_CONSEC_FRAMES_ALERT;
            
            

        # draw the total number of blinks on the frame along with
        # the computed eye aspect ratio for the frame
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
#         cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        if alert:
            cv2.putText(frame, "ALERT", (700, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


    # show the frame
    cv2.imshow("Frame", frame)
    downloader.putFrame(frame);
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break


# do a bit of cleanup
downloader.stop();
# vs.stop()
capture.release();
cv2.destroyAllWindows()

processing frame 25 out of inf
processing frame 50 out of inf
processing frame 75 out of inf
processing frame 100 out of inf
processing frame 125 out of inf
processing frame 150 out of inf
processing frame 175 out of inf
processing frame 200 out of inf
processing frame 225 out of inf
processing frame 250 out of inf
processing frame 275 out of inf
processing frame 300 out of inf
processing frame 325 out of inf
processing frame 350 out of inf
processing frame 375 out of inf
processing frame 400 out of inf
